In [1]:
import numpy as np
import gym
import d4rl

No module named 'flow'
No module named 'dm_control'
No module named 'carla'
pybullet build time: Mar 26 2022 03:00:52


# Extract Trajectory from Dataset

In [2]:
def get_trajectory(env, terminate_on_end=False, **kwargs):

    dataset = env.get_dataset(**kwargs)

    N = dataset['rewards'].shape[0]
    
    # The newer version of the dataset adds an explicit
    # timeouts field. Keep old method for backwards compatability.
    use_timeouts = False
    if 'timeouts' in dataset:
        use_timeouts = True

    episode_step = 0
    start_idx, data_idx = 0, 0
    trj_idx_list = []
    for i in range(N-1):
        done_bool = bool(dataset['terminals'][i])

        if use_timeouts:
            final_timestep = dataset['timeouts'][i]
        else:
            final_timestep = (episode_step == env._max_episode_steps - 1)
        if (not terminate_on_end) and final_timestep:
            # Skip this transition and don't apply terminals on the last step of an episode
            episode_step = 0
            trj_idx_list.append([start_idx, data_idx-1])
            start_idx = data_idx
            continue  
        if done_bool or final_timestep:
            episode_step = 0
            trj_idx_list.append([start_idx, data_idx])
            start_idx = data_idx + 1
            
        episode_step += 1
        data_idx += 1
        
    trj_idx_list.append([start_idx, data_idx])
    
    return trj_idx_list

In [3]:
env_list= [
    'walker2d-medium-expert-v2', 
    'walker2d-medium-replay-v2', 
    'walker2d-medium-v2', 
    'walker2d-expert-v2', 
    'halfcheetah-medium-v2', 
    'halfcheetah-expert-v2', 
    'halfcheetah-medium-expert-v2', 
    'halfcheetah-medium-replay-v2', 
    'hopper-medium-v2', 
    'hopper-expert-v2', 
    'hopper-medium-expert-v2', 
    'hopper-medium-replay-v2']

In [4]:
for env_name in env_list:
    env = gym.make(env_name).unwrapped
    trj_idx_list = get_trajectory(env)

/home/kimin/anaconda3/envs/jaxcql/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 22.60it/s]
